In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Hello World: Ваша перша квантова схема

Побудуйте [стан Белла](https://en.wikipedia.org/wiki/Bell_state) (два заплутаних кубіти) і виконайте його трьома способами:

1. **Ідеальна симуляція** — ідеальні результати, не потрібен обліковий запис
2. **Шумна симуляція** — симулює реальний пристрій, не потрібен обліковий запис
3. **Реальне квантове обладнання** — потрібен [обліковий запис IBM Quantum](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)

## Побудова схеми

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw(output="mpl")

## Варіант 1: Ідеальна симуляція (не потрібен обліковий запис)
Використовує `StatevectorSampler` — локальний симулятор з ідеальними результатами без шуму.

In [ ]:
from qiskit.primitives import StatevectorSampler

result = StatevectorSampler().run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## Варіант 2: Шумна симуляція (не потрібен обліковий запис)
Використовує `FakeManilaV2` — локальний симулятор, що імітує реальний квантовий пристрій IBM, включаючи його характеристики шуму. Спочатку схему необхідно транспілювати (адаптувати) до набору вентилів пристрою та з'єднання кубітів.

In [ ]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeManilaV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Варіант 3: Реальне квантове обладнання
Потрібен обліковий запис IBM Quantum. Дивіться [Налаштування облікового запису IBM Quantum](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account) для деталей.

Якщо Ви ще не зберегли свої облікові дані в цій сесії Binder, виконайте це спершу:

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token="<your-api-key>",
    instance="<your-crn>",
    overwrite=True
)

**Примітка:** Завдання на реальному обладnanні можуть зайняти певний час залежно від черги. Якщо комірка все ще виконується, ви можете перевірити статус завдання та побачити результати на [quantum.cloud.ibm.com/workloads](https://quantum.cloud.ibm.com/workloads?user=me).

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Running on {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Що далі?
- **[Посібники](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=docs/tutorials)** — покрокові посібники з алгоритмів, зменшення помилок, транспіляції та інше
- **[Курси](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=learning/courses)** — структуровані шляхи навчання від основ квантової механіки до корисних обчислень у масштабі
- **[Режим локального тестування](https://janlahmann.github.io/Qiskit-documentation/#no-token-use-local-testing-mode)** — виконуйте більшість ноутбуків без облікового запису IBM Quantum